In [6]:
import pandas as pd, numpy as np
import json
from tqdm import tqdm

def process(input_str):
    return json.loads(input_str)

def get_label(row):
    label = [idx for idx, option in enumerate(['winner_model_a','winner_model_b','winner_tie']) if row[option] == 1]
    if label[-1] == 0:
        return 'A'
    elif label[-1] == 1:
        return 'B'
    else:
        return 'C'
    return label[-1]

def load_json(data):
    data.loc[:, 'prompt'] = data['prompt'].apply(process)
    data.loc[:, 'response_a'] = data['response_a'].apply(process)
    data.loc[:, 'response_b'] = data['response_b'].apply(process)
    return data

In [7]:
data = pd.read_csv("dataset/train.csv")
data = pd.read_csv('dataset/lmsys-chatbot_arena_conversations-33k.csv')

In [5]:
data.prompt.values[0]

"['What is the difference between OpenCL and CUDA?']"

In [ ]:
data = load_json(data)

In [30]:
data_new

,id,model_a,model_b,winner_model_a,winner_model_b,winner_tie,prompt,response_a,response_b
0,58210e39b3fd4441a2bd4a518bb44c2d,chatglm-6b,koala-13b,0,1,0,['What is the difference between OpenCL and CU...,['OpenCL and CUDA are two different programmin...,['OpenCL and CUDA are both programming languag...
1,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,0,0,1,['Why did my parent not invite me to their wed...,['It is possible that your parent did not invi...,['It is likely that they wanted to keep the gu...
2,90bfd142157948aba01931726c888e7f,koala-13b,oasst-pythia-12b,0,1,0,"['Fuji vs. Nikon, which is better?']",['Both Fuji and Nikon are popular camera brand...,"[""This is a subjective question and the answer..."
3,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,0,1,0,['How to build an arena for chatbots?'],['Building an arena for chatbots can be done b...,['Building an arena for chatbots is a great wa...
4,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,1,0,0,['When is it today?'],"[""I'm sorry, I cannot determine the current da...","['Today is February 23, 2023.']"
...,...,...,...,...,...,...,...,...,...
32995,8777c4945d85469d96cd26fc2ea6f64a,alpaca-13b,claude-instant-v1,0,0,1,['who is the president of the U.S.A?'],['Joe Biden is currently the President of the ...,['Joe Biden is currently the 46th president of...
32996,86063a921be548989c55b85497ab009a,claude-instant-v1,guanaco-33b,1,0,0,['how to train lora for stable diffusion? expl...,"[""Training Stable Diffusion models like LoRA r...","[""Lora is a machine learning model that is use..."
32997,6685a3b3863f4554887e432f7dbbe8a5,wizardlm-13b,oasst-pythia-12b,0,1,0,['남녀 섹스 체위 자세 10가지를 적어줘'],['1. 웨이퍼 에폭보: 남녀 섹스로 웨이퍼 에폭보는 이미 입문으로 이루어져 있는 ...,"['1.\t""섹스""\n2.\t""체""\n3.\t""위""\n4.\t""자""\n5.\t""세""..."
32998,f72930b382e949ea879e7abf3cb1e587,guanaco-33b,koala-13b,1,0,0,['how to evaluate a language model output?'],"[""Evaluating a language model output involves ...","[""Evaluating the output of a language model in..."


In [4]:
data = data.explode(['prompt','response_a','response_b']).reset_index(drop = True)

In [5]:
data['label'] = data.apply(lambda x: get_label(x), axis = 1)

In [6]:
data = data.fillna('None')

In [7]:
data['response_a'] = data['response_a'].apply(lambda x: 'None' if len(x)==0 else x)
data['response_b'] = data['response_b'].apply(lambda x: 'None' if len(x)==0 else x)

In [ ]:
#prompt1
'''
#Model A
Prompt1: xxx
Response: xxx

Prompt2: xxx
Response: xxx

#Model B
Prompt1: xxx
Response: xxx

Prompt2: xxx
Response: xxx
'''

In [ ]:
data['prompt_response_A'] = "Prompt: " + data['prompt'] + "\n" + "Response: " + data['response_a']
data['prompt_response_B'] = "Prompt: " + data['prompt'] + "\n" + "Response: " + data['response_b']
data = data.groupby('id').agg({'prompt_response_A': '\n\n'.join, 'prompt_response_B': '\n\n'.join, 'label': lambda x: list(x)[0]}).reset_index()
data['prompt_response'] = "#Model A\n" + data['prompt_response_A'] + "\n\n#Model B\n" + data['prompt_response_B']

In [ ]:
print(data.prompt_response.values[0])

In [ ]:
#prompt2
'''
Label A
#prompt1
xxxx
#Response
##Model A
##Model B

到这里超过max length了，另外变成一行，Label还是A

#Prompt2
xxxxx
#Response
##Model A
xxxx
##Model B
xxxx
'''


In [8]:
data['prompt_response'] = "#Prompt\n" + data['prompt'] + "\n\n" + "#Response\n" + "##Model A\n" + data['response_a'] + "\n\n" + "##Model B\n" + data['response_b']

In [9]:
max_length = 3000

In [10]:
prompt_response = []
ids = []
labels = []
text_length = 0
for idx, row in tqdm(data.iterrows(), total=len(data)):
    text = row['prompt_response']
    label = row['label']
    id = row['id']
    if id not in ids:
        #第一次出现
        prompt_response.append(text)
        text_length = len(text.split(" "))
        ids.append(id)
        labels.append(label)
    else:
        text_length += len(text.split(" "))
        if text_length <= max_length:
            #print(f"before: {prompt_response[-1]}")
            #取上一个text出来，合并后替换
            text = prompt_response[-1] + "\n\n" + text
            prompt_response[-1] = text
            #print(f"after: {prompt_response[-1]}")
        else:
            #另一起一行
            prompt_response.append(text)
            text_length = len(text.split(" "))
            ids.append(id)
            labels.append(label)
    

100%|██████████| 71514/71514 [00:19<00:00, 3669.76it/s]


In [11]:
length = [len(i.split(" ")) for i in prompt_response]

In [14]:
length.index(21579)

42803

In [15]:
ids[42803]

3169432118

In [12]:
max(length)

21579

In [ ]:
# 天才问题
# id = 3169432118

In [18]:
data = pd.DataFrame({'id': ids, 'prompt_response': prompt_response, "labels": labels})

In [ ]:
data.id.value_counts()

In [19]:
tmp = data.loc[data.id == 3169432118,:]
tmp

,id,prompt_response,labels
42803,3169432118,#Prompt\nCreate an ascii map of a fantasy land...,A


In [ ]:
print(tmp.response_b.values[-2])

In [ ]:
print(tmp.prompt_response.values[-1])

In [2]:
from utils import load_spilt_data
data_path = 'dataset/train.csv'
prompt_type = 2
MAX_INPUT = 3000
if_train = True
df_train , df_valid = load_spilt_data(data_path, prompt_type, MAX_INPUT, if_train)

100%|██████████| 71514/71514 [00:18<00:00, 3783.58it/s]


In [ ]:
print(df_train.prompt_response.values[0])

In [ ]:
t = data.loc[data.id == 470485337]
print(t.response_a.values[0])

In [7]:
t = df_train.loc[df_train.id == 470485337]

In [ ]:
print(t.prompt_response.values[0])